In [8]:
import re
import spacy
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
tqdm.pandas()

In [2]:
def clean_text(input_text):    
    input_text = str(input_text)
    
    # Проверка на рекламу по ЕРИР
    ad = r'Реклама.*(ERID|Erid|erid|ОГРН)'

    if bool(re.search(ad, input_text)):
        return "РЕКЛАМА"
    
    # Проверка на рекламу по хэштегам
    # Список хэштегов для проверки
    hashtags_to_check = ['#нативнаяинтеграция', '#интеграция', '#реклама', '#спонсор']

    # Регулярное выражение для поиска хэштегов в тексте
    pattern = '|'.join(re.escape(tag) for tag in hashtags_to_check)

    # Проверяем, содержит ли текст один из хэштегов
    contains_hashtags = bool(re.search(pattern, input_text))

    if contains_hashtags:
        return "РЕКЛАМА"

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', input_text)
    
    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()
        
    clean_text = clean_text.replace('ё', 'е')
    clean_text = clean_text.replace('Ё', 'Е')

    # Заменяем перенос строки на точку
    clean_text = re.sub('\n', '. ', clean_text)
    clean_text = re.sub('\n\n', '. ', clean_text)

    # Убираем все лишние пробелы и тчки
    clean_text = re.sub('(\. )+', '. ', clean_text)
    
    # Убираем специальные символы
    clean_text = re.sub('[^а-яА-Яa-zA-Z]', ' ', clean_text)
    clean_text = re.sub('\s+', ' ', clean_text)

    return clean_text

In [3]:
def get_lemma(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

In [3]:
def sw(clean_text):
    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    return clean_text

In [4]:
nlp = spacy.load('ru_core_news_lg', exclude=['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'ner'])

In [20]:
df = pd.read_csv('../../raw_data/mads_news_27cat.csv')

In [5]:
df['clear_text'] = df['text'].apply(clean_text)

In [6]:
df = df.query('clear_text != "РЕКЛАМА"')

In [8]:
df['clear_text_doc'] = list(nlp.pipe(df['clear_text'], n_process=-1))

In [9]:
df['clear_text'] = df['clear_text_doc'].apply(get_lemma)

In [11]:
df.sample(5)

,Unnamed: 0,date,text,category,name,channel_id,clean_text,clear_text
0,262,2021-09-09 13:50:22,​Что требуется для успеха в переговорах.\n\nЛу...,Бизнес и стартапы,Бизнес Стиль,1482665221,что требуется успеха переговорах . лучшие пере...,требуется успеха переговорах лучшие переговорщ...
1,248,2021-08-31 10:04:44,​«Сосредоточь все силы на достижение желаемого...,Бизнес и стартапы,Бизнес Личность,1169845777,"сосредоточь силы достижение желаемого , врем...",сосредоточь силы достижение желаемого времени ...
2,493,2022-06-22 18:14:44,​Самомотивация или как мотивировать себя. 6 ко...,Бизнес и стартапы,Бизнес Стиль,1482665221,самомотивация мотивировать . 6 конкретных прие...,самомотивация мотивировать конкретных приемов ...
3,864,2022-02-25 10:09:00,"Бизнес-кейс: Создание сложных сайтов, CRM\n\n💸...",Бизнес и стартапы,Business Advisor,1284946539,"бизнес-кейс создание сложных сайтов , crm одн...",бизнес кейс создание сложных сайтов crm одной ...
4,570,2022-10-17 20:43:18,"​​3 эксперимента, применимых в бизнесе.\n\nПси...",Бизнес и стартапы,Бизнес Стиль,1482665221,"3 эксперимента , применимых бизнесе . психолог...",эксперимента применимых бизнесе психологически...
...,...,...,...,...,...,...,...,...
81022,132,2022-12-26 14:25:07,"Тай. Часть 3.\n\nБез приключений, как вы поним...",Блог,Семья и Я,1693400962,"тай . часть 3. приключений , понимаете , обошл...",тай часть приключений понимаете обошлось прямо...
81023,133,2022-12-31 15:06:10,"2️⃣ 0️⃣ 2️⃣ 3️⃣\n\nДорогие мои, сегодня послед...",Блог,Семья и Я,1693400962,"2 0 2 3 дорогие мои , сегодня последний день у...",дорогие мои сегодня последний день уходящего г...
81024,134,2023-01-06 17:52:58,Новогодние праздники прошли довольно насыщено....,Блог,Семья и Я,1693400962,новогодние праздники прошли довольно насыщено ...,новогодние праздники прошли довольно насыщено ...
81025,135,2023-01-10 13:32:33,💰Ёмкость финансовой чаши.\n\n\nНашёл в простор...,Блог,Семья и Я,1693400962,емкость финансовой чаши . нашел просторах сети...,емкость финансовой чаши нашел просторах сети т...
